# Voting

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import f1_score, precision_score, accuracy_score,\
                            recall_score, roc_curve, auc, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

sns.set()

df_data = pd.read_csv('https://drive.google.com/uc?export=download&id=1i-KJ2lSvM7OQH0Yd59bX01VoZcq8Sglq')
df_decision = pd.read_csv('https://drive.google.com/uc?export=download&id=1km-AEIMnWVGqMtK-W28n59hqS5Kufhd0')

## Preprocesado

In [2]:
from Preprocessing import *

In [3]:
df = preprod_tp1(df_data, df_decision)
df = replace_nulls_edad(df, 'media')
df = encodear_atributos_categoricos(df)
df.head()

,tipo_de_sala_0,tipo_de_sala_1,tipo_de_sala_2,id_usuario,genero_0,genero_1,edad,amigos,parientes,precio_ticket,nombre_sede_0,nombre_sede_1,nombre_sede_2,cant_acompañantes,volveria
0,0,0,1,117,0,1,73.5,0,0,1,0,0,1,0,0
1,0,0,1,658,1,0,35.0,1,1,2,0,0,1,2,0
2,0,1,0,794,0,1,32.7,0,0,3,0,1,0,0,0
3,0,0,1,455,0,1,32.7,0,0,1,0,1,1,0,0
4,0,0,1,173,1,0,4.0,1,1,2,0,1,1,2,1


## Entrenamiento

In [4]:
X, y = df.iloc[:,:-1], df.iloc[:,-1]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=117)

### Entrenamiento

In [6]:
#bagging = GradientBoostingClassifier(random_state=117, n_estimators=98, min_samples_split=9,
#                                    max_depth=6, loss='exponential', learning_rate=0.2, criterion='mse')

In [13]:
arbol  = DecisionTreeClassifier(max_depth=3, min_samples_leaf=10,random_state=117, criterion="entropy")

In [16]:
forest = RandomForestClassifier(n_estimators=11, min_samples_split=19, max_samples=46, 
                                max_features=2, criterion='entropy',random_state=117)

In [17]:
ada = AdaBoostClassifier(
        base_estimator = DecisionTreeClassifier(max_depth=3, min_samples_leaf=10,random_state=117, criterion="entropy"),
    random_state=117, n_estimators=61, learning_rate=0.2, algorithm='SAMME')

In [30]:
model_voting = VotingClassifier(
        estimators=[('arbol', arbol), ('forest', forest), ('ada', ada)], voting='soft'
    )

In [31]:
arbol.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=10,
                       random_state=117)

In [32]:
forest.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_features=2, max_samples=46,
                       min_samples_split=19, n_estimators=11, random_state=117)

In [33]:
ada.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=3,
                                                         min_samples_leaf=10,
                                                         random_state=117),
                   learning_rate=0.2, n_estimators=61, random_state=117)

In [34]:
model_voting.fit(X_train, y_train)

VotingClassifier(estimators=[('arbol',
                              DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=3,
                                                     min_samples_leaf=10,
                                                     random_state=117)),
                             ('forest',
                              RandomForestClassifier(criterion='entropy',
                                                     max_features=2,
                                                     max_samples=46,
                                                     min_samples_split=19,
                                                     n_estimators=11,
                                                     random_state=117)),
                             ('ada',
                              AdaBoostClassifier(algorithm='SAMME',
                                                 base_estimator=DecisionTreeClassifier(criterion='e

In [37]:
prob_pred_voting = arbol.predict_proba(X_test)

In [38]:
accuracy_score(prob_pred_voting, y_test)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets